<a href="https://colab.research.google.com/github/giuoaejgiusejnb/mhrsb-qurious-crafting-ocr-kotlin-app/blob/main/notebooks/qurious_crafting_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# スキルデータを作成

In [ ]:
import requests
import json

url = 'https://raw.githubusercontent.com/giuoaejgiusejnb/mhrsb-qurious-crafting-ocr-kotlin-app/refs/heads/main/shared-assets/all-skills.json'
response = requests.get(url)

# 練成で出るスキルをコスト順に管理
SKILL_DATA = response.json()["skill_data"]

# 全スキルを集めたリスト
SKILL_MASTER_LIST = [skill for skills in SKILL_DATA.values() for skill in skills]

#　pip

In [ ]:
!pip install easyocr

# # importと定数定義

In [ ]:
import cv2
import difflib
import easyocr
import glob
import json
import numpy as np
import os
import time
import zipfile
from pathlib import Path
import gradio as gr
import shutil
import torch

INPUT_ZIP_FILE = "mhrsb_temp.zip"
UNZIPED_DIR = "unzipped"
SETTINGS_JSON_PATH = f"{UNZIPED_DIR}/temp_settings.json"
FILTERD_DIR = "results"
FILTERD_ZIP_DIR = FILTERD_DIR + ".zip"

# google driveにマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ocr処理を行う関数

In [ ]:
def correct_skill_name(ocr_text, master_list):
    """
    OCRテキストをマスターリスト内の最も近いスキル名に補正します。
    """
    # get_close_matches: 最も似ている候補をリストで返す
    # 0.45は小さすぎるかも
    matches = difflib.get_close_matches(ocr_text, master_list, n=1, cutoff=0.45)
    if matches:
        # 最も似ている候補を返す
        return matches[0]
    else:
        # 適切な候補が見つからない場合は該当なしと返す
        return "該当なし"

def detect_skills_easyocr(image_paths, allowlist):
    # 1. リーダーは関数の外、またはループの直前で1回だけ初期化
    reader = easyocr.Reader(['ja', 'en'], gpu=True)
    image_skills_list = []

    if not os.path.exists(FILTERD_DIR):
        os.makedirs(FILTERD_DIR)

    total_images = len(image_paths)
    CHUNK_SIZE = 12

    # 2. まとめて読み込まず、必要な分だけその都度読み込む
    for i in range(0, total_images, CHUNK_SIZE):
        batch_paths = image_paths[i : i + CHUNK_SIZE]
        current_chunk_images = []

        for p in batch_paths:
            img = cv2.imread(p)
            if img is not None:
                roi = img[361: 565, 540: 700]
                roi = cv2.resize(roi, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                current_chunk_images.append(roi)

        # OCR実行

        results_chunk = reader.readtext_batched(
                current_chunk_images, allowlist=allowlist, batch_size=4, detail=0, paragraph=False
            )

        for j, results in enumerate(results_chunk):
            file_idx = i + j
            skills = [correct_skill_name(text.strip(), SKILL_MASTER_LIST)
            for text in results]
            image_skills_list.append([image_paths[file_idx], skills])

        # 3. 非常に重要：VRAMの定期解放
        torch.cuda.empty_cache()

        if i % 120 == 0: # 120枚ごとに進捗表示
             gr.Info(f"OCR実行中... ({i}/{total_images})")

    return image_skills_list


def filter_images_by_skills(image_paths):
    # jsonファイルを読み込み，欲しいスキルを集めたリストを作成
    with open(SETTINGS_JSON_PATH, "r", encoding="utf-8") as f:
        desired_skills = json.load(f)
    unique_chars = "".join((list(set("".join(SKILL_MASTER_LIST)))))
    gr.Info("画像認識開始")
    for file_no, image_skills_pairs in enumerate(detect_skills_easyocr(image_paths, unique_chars), 1):
        image_path = image_skills_pairs[0]
        skills = image_skills_pairs[1]
        for skill in skills:
            if skill in desired_skills:
                cv2.imwrite(f'results/{str(file_no)}.jpg', cv2.imread(image_path))
                break

# ocrを開始し，ダウンロードボタン表示

In [ ]:
def run_ocr():
  # Googleドライブ全体からzipファイルを検索
  target_file = INPUT_ZIP_FILE
  found = False
  for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if target_file in files:
      full_path = os.path.join(root, target_file)
      zip_file = full_path
      found = True
      break

  if not found:
    raise gr.Error(f"エラー: 指定されたファイル '{target_file}' が見つかりません！")

  gr.Info("zipファイルを解凍中")
  extract_dir = UNZIPED_DIR
  with zipfile.ZipFile(zip_file, 'r') as zf:
    zf.extractall(path=extract_dir)
  image_paths = glob.glob(os.path.join(extract_dir, "**/*.jpg"), recursive=True)
  image_paths.sort()

  # ocr 実行
  filter_images_by_skills(image_paths)
  # 圧縮フェーズ
  gr.Info("ファイルを生成しています")
  output_zip_path = shutil.make_archive(
      Path(FILTERD_ZIP_DIR).stem,
      format='zip',
      root_dir=FILTERD_DIR
      )

  #完了
  gr.Info("処理終了")
  # ボタンを表示し、ファイルをセット
  return gr.update(value=FILTERD_ZIP_DIR, visible=True), gr.update(visible=True)

def delete_drive_file():
    file_path = "/content/drive/MyDrive/" + INPUT_ZIP_FILE

    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            return f"driveの{file_path}を削除しました"
        except Exception as e:
            return f"エラー: {str(e)}"
    else:
        return "エラー: ファイルが見つかりません"

with gr.Blocks() as demo:
    gr.Markdown("### 処理が終わったら下にダウンロードボタンが出ます")

    # 最初は隠しておく
    dl_btn = gr.DownloadButton("生成されたファイルをダウンロード", visible=False)
    delete_btn = gr.Button("driveにアップデートしたzipファイルを削除", variant="stop", visible=False)
    status_output = gr.Textbox(show_label=False)
    delete_btn.click(fn=delete_drive_file, outputs=status_output)

    # ページ読み込み時（load）に関数 f を実行
    demo.load(fn=run_ocr, inputs=None, outputs=[dl_btn, delete_btn])

demo.launch()
